## Check the signature of the generated model

saved_model_cli show --dir tfx_pipeline/pipeline_tfx/serving_model/pipeline_tfx/1619546428/ --tag_set serve --signature_def serving_default

In [ ]:
import base64
import requests
import pandas as pd
import tensorflow as tf

SERVING_MODEL_DIR = "../tfx_pipeline/pipeline_tfx/serving_model/pipeline_tfx/1619546428/"
loaded_model = tf.keras.models.load_model(SERVING_MODEL_DIR)
inference_fn = loaded_model.signatures['serving_default']

In [ ]:
LABEL_KEY = "close_t"
df_train = pd.read_csv(f"../data/raw/train_data/train_data.csv")
FEATURE_KEYS = list(df_train.columns.values)
if LABEL_KEY in FEATURE_KEYS:
    FEATURE_KEYS.remove(LABEL_KEY)
    values = df_train.iloc[0:1000,:]    
    values = values.drop([LABEL_KEY], axis=1).values
else:
    values = df_train.iloc[16:20,:]

In [ ]:


for row in values:
    features = {}

    for column, value in zip(FEATURE_KEYS, row):
        features[column] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

    example_proto = tf.train.Example(features=tf.train.Features(feature=features))
    examples = example_proto.SerializeToString()

    result = inference_fn(examples=tf.constant([examples]))
    print(result['output_0'].numpy())



In [ ]:
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

## Serving model in docker 

    docker run --name tf_trading \
        -p 8501:8501 \
        -v "$(pwd)/tfx_pipeline/pipeline_tfx/serving_model/pipeline_tfx:/models/my_model" \
        -e MODEL_NAME=my_model \
        -t tensorflow/serving

    GET http://localhost:8501/v1/models/my_model

    GET http://localhost:8501/v1/models/my_model/metadata

## Make predictions with serving model

In [ ]:
def _bytes_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value.encode()])
        )

def _float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value])
        )

def _int64_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value])
        )

In [ ]:
example = tf.train.Example(
    features=tf.train.Features(
        feature=features
    )
)

serialized_example = example.SerializeToString()
prediction = inference_fn(tf.constant([serialized_example]))
prediction

In [ ]:
example_bytes = base64.b64encode(serialized_example).decode('utf-8')
payload = {
    'signature_name': 'serving_default',
    'instances': [{
        'b64': example_bytes
    }]
}

model_server_url = "http://localhost:8501/v1/models/my_model:predict"
resp = requests.post(model_server_url, json=payload)
print(resp.json()) 

GET http://localhost:8501/v1/models/my_model

GET http://localhost:8501/v1/models/my_model/metadata

